In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

%matplotlib inline 
import matplotlib as mpl
mpl.rcParams['figure.dpi']
import graphviz

In [14]:
df=pd.read_csv('train.csv')
tf=pd.read_csv('test.csv')
tf=tf.columns.tolist()
features_response=df.columns.tolist()
items_to_remove=['PassengerId']
features_response = [item for item in features_response if item not in items_to_remove ]
features_response

['Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import tree

X_train,X_test,y_train,y_test = train_test_split(df[features_response[:-1]].values,tf.values,test_size=0.2,random_state=24)

AttributeError: 'list' object has no attribute 'values'